O mercado imobiliário brasileiro tem passado por um momento de aumento de preços nos últimos anos, puxados pelo aumento da taxa básica de juros (Selic) e pelo custo do material da construção civil, e esse fenômeno não ocorre apenas no Brasil, os preços dos imóveis dispararam pelo mundo. Além dos motivos supramencionados, outro fator que tem forte impacto nesse. 

Enquanto os donos dos imóveis tem muitas razões para comemorar, o mesmo não ocorre com quem está em busca de um apartamento para comprar ou alugar.

Além dos motivos supramencionados, destacam-se outros fatores, tais como:
"Os preços das moradias estão subindo por causa da pandemia, não apesar dela", diz Kate Everett-Allen, chefe de Pesquisa Residencial Internacional da Knight Frank, à BBC News Mundo, o serviço de notícias em espanhol da BBC. Com as profundas mudanças geradas a partir de 2020, explica ela, se gerou uma reavaliação maciça das necessidades de moradia das pessoas. "Esta é uma corrida por espaço. As pessoas estão comprando como loucas."
Referência:
https://economia.uol.com.br/noticias/bbc/2021/08/02/precos-dos-imoveis-disparam-pelo-mundo.htm

Ultimamente, o mercado de alugueis anda tão imprevisível quanto o próprio mercado de compra e venda dos mesmos. Diante disso, há a necessidade de uma pesquisa apurada em todo o montante de ofertas disponíveis para alugar com o objetivo de responder as seguintes perguntas:
- Como é o comportamento da distribuição dos preços dos imóveis disponíveis pra alugar?
- Quais as regiões mais valorizadas da cidade?
- Qual a área mais interessante pra iniciar a procura?

No segundo momento:
- Criar um modelo de recomendação baseado nas minhas preferencias.
- Criar um modelo preditivo capaz de avaliar se o apartamento (ou apartamentos) sugerido está acima ou abaixo do preço praticado no mercado. <-- Não foi aplicado

In [11]:
import glob
import json
import os
import re
import time
import warnings
from datetime import datetime

import bs4
import numpy as np
import pandas as pd
import pycep_correios as pycep
import requests as rq
import tqdm
from unidecode import unidecode

warnings.filterwarnings("ignore")

%config Completer.use_jedi = False

# Filtro utilizado para o webscapping

## Opção #01

In [12]:
query = 'filtro_centro_sp'

query = re.sub(' +', ' ', query)
query = unidecode(query.strip().lower()).replace(' ', '%20') # Trasnformacao para o formado de busca no olx
query 

'filtro_centro_sp'

In [13]:
# URL para utilizar a query (busca)
#url = 'https://sp.olx.com.br/imoveis/aluguel/apartamentos?o={page}&q={query}'

## Opção #02

In [14]:
# URL para utilizar filtro (Centro de Sao Paulo)
url = 'https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o={page}'  

In [15]:
# User- agent for get response
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

response = rq.get(url, headers= headers)
response # 200 - ok!

<Response [200]>

# Salvando as paginas em HTML das abas da pesquisa (01_raw_data_olx)

In [16]:
pages = 50 # Quantidade de paginas baixadas
source = 'olx' # Fonte dos dados  

In [17]:
try:
    now = datetime.now().strftime('%Y-%m-%d') # Data de hoje
    os.mkdir(f'./01_raw_data_olx/{now}')
except:
    pass

In [19]:
# Salvando as X paginas para scrapping offline
try:
    now = datetime.now().strftime('%Y-%m-%d') # Data de hoje
    os.mkdir(f'./01_raw_data_olx/{now}') # Criando uma pasta com a data de hoje
except:
    pass

for page in tqdm.tqdm_notebook(range(1,pages+1)):
    
    #url_ = url.format(query=query, page=page) # <-- url_ utilizando query
    url_ = url.format(page=page)
    print(f'Aba {page}:\n{url_}')
    
    response = rq.get(url_, headers= headers)
    
    with open("./01_raw_data_olx/{}/{}_aba_{}.html".format(now, query, page), 'w+', encoding='utf-8') as output:
        output.write(response.text)
    time.sleep(2)

  0%|          | 0/50 [00:00<?, ?it/s]

Aba 1:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=1
Aba 2:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=2
Aba 3:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=3
Aba 4:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=4
Aba 5:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=5
Aba 6:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=6
Aba 7:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=7
Aba 8:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=8
Aba 9:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=9
Aba 10:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=10
Aba 11:
https://sp.olx.com.br/sao-paulo-e-regiao/centro/imoveis/aluguel/apartamentos?o=11
Aba 12:
https://sp.olx.com.b

# Coletando os links de cada pagina salva (01_raw_data_olx)

In [21]:
# Scrapping nas paginas salvas
for page in tqdm.tqdm_notebook(range(1,pages+1)):
    with open('./01_raw_data_olx/{}/{}_aba_{}.html'.format(now, query, page), 'r+', encoding='utf-8') as html:
        
        now = datetime.now().strftime('%Y-%m-%d') # Data de hoje
        
        page_html = html.read()
        
        parsed = bs4.BeautifulSoup(page_html)
        
        tags = parsed.findAll("a")
        
        for tag in tags:
            if tag.has_attr('data-lurker_list_id'):
                link = tag['href']
                title = tag['title']
                cod = tag['data-lurker_list_id']

                
                with open(f"./01_parsed_links/parsed_links_{now}.json", "a+") as output:
                    data = {"link":link, # link
                           "title":title, # Titulo do anuncio
                           "query":query, # Query utilizada para busca
                           'list_id':cod, # Codigo do anuncio (unico)
                           'aqs_date':now, # Data de aquisicao dos dados
                           'source':source}
                           #'thumb': thumb} # Fonte dos dados (olx)
                    output.write("{}\n".format(json.dumps(data)))

  0%|          | 0/50 [00:00<?, ?it/s]

## Verificação do resultado e processando os dados

In [22]:
df_1 = pd.read_json(f'./01_parsed_links/parsed_links_{now}.json', lines=True)
df_1.tail()

,link,title,query,list_id,aqs_date,source
2495,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,Ótimo apartamento de 42 m² com sacada para loc...,filtro_centro_sp,950600339,2021-11-10,olx
2496,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,Studio de 23 m² com sacada,filtro_centro_sp,950600342,2021-11-10,olx
2497,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,Studio de 22 m² com sacada,filtro_centro_sp,950600351,2021-11-10,olx
2498,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,Studio de 23 m² com sacada,filtro_centro_sp,950600359,2021-11-10,olx
2499,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,Studio de 22 m² com sacada,filtro_centro_sp,950600365,2021-11-10,olx


In [23]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   link      2500 non-null   object
 1   title     2500 non-null   object
 2   query     2500 non-null   object
 3   list_id   2500 non-null   int64 
 4   aqs_date  2500 non-null   object
 5   source    2500 non-null   object
dtypes: int64(1), object(5)
memory usage: 117.3+ KB


In [24]:
# Quantidade de dados salvos
print(f'{len(df_1)} dados salvos')

2500 dados salvos


In [25]:
# Quantidade de dados duplicados
#dup_itens = df_1.duplicated(['list_id', 'aqs_date'])
dup_itens = df_1.duplicated(['list_id'])
print(f'{dup_itens.sum()} dados duplicados')
display(df_1[dup_itens].head())

0 dados duplicados


,link,title,query,list_id,aqs_date,source


In [26]:
# Deletando dados duplicados, se aplicavel
df_1.drop_duplicates(subset=['list_id', 'aqs_date'], keep="last", inplace=True)

# Quandidade de dados depois do drop
print(f'{len(df_1)} dados restante depois do drop')

2500 dados restante depois do drop


# Salvando a pagina de cada anuncio

In [29]:
try:
    now = datetime.now().strftime('%Y-%m-%d') # Data de hoje
    os.mkdir(f'./02_raw_data_olx_itens/{now}') # Criando um diretorio com o dia de hoje
except:
    pass

In [31]:
# Looping para salvar as paginas de cada anuncio


links_list = df_1['link']

i = 1
for link in tqdm.tqdm_notebook(links_list):
    url_ = link
    print(f'Item {i}:\n{url_}')
    
    response = rq.get(url_, headers=headers)
    
    link_name = re.search('(\d+)(?!.*\d)', link).group() # Captura o código de cada anuncio

    with open("./02_raw_data_olx_itens/{}/item_cod_{}.html".format(now, link_name), 'w+', encoding='utf-8') as output:
        output.write(response.text)
    time.sleep(1)
    i  += 1

  0%|          | 0/2500 [00:00<?, ?it/s]

Item 1:
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/aluga-se-apto-no-centro-sp-955129983
Item 2:
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/excelente-apartamento-aclimacao-todo-reformado-955121409
Item 3:
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/apartamento-residencial-para-locacao-3-dts-vaga-de-garagem-r-2900-00-945892190
Item 4:
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/aluga-se-kitnet-na-regiao-do-bras-sem-analise-de-credito-929533029
Item 5:
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/apartamento-para-aluguel-possui-160-metros-quadrados-com-3-quartos-em-santa-cecilia-sao-935621122
Item 6:
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/kitnet-confortavel-e-aconchegante-prox-metro-944239390
Item 7:
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/amplo-apartamento-ao-lado-do-parque-da-aclimacao-e-avenida-lins-de-vasconcelos-59-m-uteis-922070493
Item 8:
https://sp.olx.com.br/sao-paulo-e-regiao/imoveis/apartamento-para-aluguel-tem-22-metros-quadrados-co

# Coletando os dados dos links de cada item

## Scratch

## Full 

In [32]:
with open(f"./02_parsed_itens_info/parsed_itens_info_{now}.json", 'w+', encoding='utf-8') as output:
    for item in tqdm.tqdm_notebook(sorted(glob.glob(f"./02_raw_data_olx_itens/{now}/item_cod*"))):
        try:
            with open(item, 'r+', encoding='utf-8') as html:
                page_html = html.read()
            
                parsed = bs4.BeautifulSoup(page_html, 'html.parser')
                item_info = parsed.find_all(string=re.compile("window.dataLayer"))
                
                desc = parsed.find_all("span", {"class": "sc-1sj3nln-1 eOSweo sc-ifAKCX cmFKIN"})
                desc = str(desc[0].text)
                
        except Exception as e: 
            print(e)
            print(item)
            continue
            
        data = dict()
        for i in item_info[0].split(',"'):
            try:
                key = i.split(':')[0].replace('"', '').strip()
                value = i.split(':')[1].replace('"', '').replace('}', '').replace('{', '').replace(']', '').strip()
                key = unidecode(key).lower()
                value = unidecode(value).lower()
            
                data[key] = value
                
            except:
                continue
                
        data['aqs_date'] = now
        data['desc'] = desc
        
        with open(f"./02_parsed_itens_info/parsed_itens_info_{now}.json", "a+") as output:
            output.write("{}\n".format(json.dumps(data)))

  0%|          | 0/2500 [00:00<?, ?it/s]

list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_906585381.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_914787379.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_922161605.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_922161760.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_922986992.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_922987041.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_926514114.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_926928016.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_931584289.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_935421546.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_935884055.html
list index out of range
./02_raw_data_olx_itens/2021-11-10\item_cod_937909157.html
list

In [33]:
df_2 = pd.read_json(f'./02_parsed_itens_info/parsed_itens_info_{now}.json', lines=True)
df_2.head()

,window.datalayer = [{page,detail,state_id,list_id,parent_category_id,category_id,city_id,zipcode,price,olxpay,...,userid,pagetype,abtestingenable,listid,site,source,aqs_date,desc,condominio,size
0,pagetype,addate,11,625553376,1000,1020,3370,1502001,2700,,...,null,ad_detail,1,625553376,ismobile,web,2021-11-10,"Apto c/ 2 dormitórios, sendo 1 dorm c/guarda-r...",NaN,NaN
1,pagetype,addate,11,639983350,1000,1020,3370,1502001,1500,,...,null,ad_detail,1,639983350,ismobile,web,2021-11-10,"kitnet na Glicério Valor a partir de R$ 1.500,...",NaN,NaN
2,pagetype,addate,11,642515452,1000,1020,3370,1306001,3300,,...,null,ad_detail,1,642515452,ismobile,web,2021-11-10,"Código do anúncio: 113422\nPé direito alto, ja...",r$ 708,110.0
3,pagetype,addate,11,668386502,1000,1020,3370,1502001,1500,,...,null,ad_detail,1,668386502,ismobile,web,2021-11-10,"Kit com sala grande, banheiro, sem divisórias,...",NaN,NaN
4,pagetype,addate,11,671376395,1000,1020,3370,1150000,4900,,...,null,ad_detail,1,671376395,ismobile,web,2021-11-10,Código do anúncio: SC4364\nCobertura na Santa ...,r$ 1.900,330.0


# Juntando os dataframes (links com os itens)

In [34]:
df_full = pd.merge(df_1, df_2, how = 'left', on = ['list_id', 'aqs_date']) # df_1: dataframe com os dados com os links(aba), df_2: informacoes dentro de cada link
df_full.head()

,link,title,query,list_id,aqs_date,source_x,window.datalayer = [{page,detail,state_id,parent_category_id,...,session,userid,pagetype,abtestingenable,listid,site,source_y,desc,condominio,size
0,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,Aluga-se Apto no Centro SP,filtro_centro_sp,955129983,2021-11-10,olx,pagetype,addate,11.0,1000.0,...,user,null,ad_detail,1.0,955129983.0,ismobile,web,Apto super arejado e iluminado pois fica no 12...,r$ 560,57.0
1,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,"excelente apartamento aclimacao , todo reformado,",filtro_centro_sp,955121409,2021-11-10,olx,pagetype,addate,11.0,1000.0,...,user,null,ad_detail,1.0,955121409.0,ismobile,web,"amplo apartamento todo reformado, 01 vaga, bem...",r$ 250,80.0
2,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,"Apartamento Residencial para locação, 3 Dts, V...",filtro_centro_sp,945892190,2021-11-10,olx,pagetype,addate,11.0,1000.0,...,user,null,ad_detail,1.0,945892190.0,ismobile,web,Código do anúncio: 9936\nApartamento Residenci...,r$ 650,104.0
3,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,Aluga-se kitnet na região do Brás ( sem analis...,filtro_centro_sp,929533029,2021-11-10,olx,pagetype,addate,11.0,1000.0,...,user,null,ad_detail,1.0,929533029.0,ismobile,web,ALUGA-SE KITNET'S SEM ANÁLISE DE CRÉDITO E 2 M...,NaN,27.0
4,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,Apartamento para aluguel possui 160 metros qua...,filtro_centro_sp,935621122,2021-11-10,olx,pagetype,addate,11.0,1000.0,...,user,null,ad_detail,1.0,935621122.0,ismobile,web,Código do anúncio: SH-AL-HEL\n\nO apartamento ...,r$ 1.700,160.0


In [35]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   link                        2500 non-null   object 
 1   title                       2500 non-null   object 
 2   query                       2500 non-null   object 
 3   list_id                     2500 non-null   int64  
 4   aqs_date                    2500 non-null   object 
 5   source_x                    2500 non-null   object 
 6   window.datalayer = [{page   2415 non-null   object 
 7   detail                      2415 non-null   object 
 8   state_id                    2415 non-null   float64
 9   parent_category_id          2415 non-null   float64
 10  category_id                 2415 non-null   float64
 11  city_id                     2415 non-null   float64
 12  zipcode                     2415 non-null   float64
 13  price                       2415 

In [36]:
# Quantidade de dados salvos
print(f'{len(df_full)} dados salvos')

2500 dados salvos


In [37]:
# Quantidade de dados duplicados
dup_itens = df_full.duplicated(['list_id', 'aqs_date', 'link'])
print(f'{dup_itens.sum()} dados duplicados')
display(df_full[dup_itens].head())

0 dados duplicados


,link,title,query,list_id,aqs_date,source_x,window.datalayer = [{page,detail,state_id,parent_category_id,...,session,userid,pagetype,abtestingenable,listid,site,source_y,desc,condominio,size


In [38]:
# Deletando dados duplicados, se aplicavel
df_full.drop_duplicates(subset=['list_id', 'aqs_date'], keep="first", inplace=True)

# Quandidade de dados depois do drop
print(f'{len(df_full)} dados restante depois do drop')

2500 dados restante depois do drop


In [39]:
df_full.shape

(2500, 52)

In [40]:
now_2 = datetime.now().strftime("%m-%d-%Y_%H%M%S") # Data e horario
df_full.to_csv(f'./data/01_raw_df/raw_df_{now_2}.csv', sep='|',  index=False) # Cria o dataframe